In [1]:
import ee
import geemap
from datetime import datetime, timedelta
import math
import matplotlib.pyplot as plt
import numpy as np
import ipynb
import time

# Get authentication credentials to initialize ee
ee.Authenticate()
ee.Initialize()

Enter verification code: 4/1AfgeXvsbDr0ySWXommj2KVWxOBOWYN0zVPbdVFNiyuXca1so2kEeeAevwPI

Successfully saved authorization token.


In [2]:
# Define geometry for data collection
shp_file = 'data/smaller_region.shp'
small_boundary = geemap.shp_to_ee(shp_file)

In [7]:
# Import era5 winds data

d1 = ['2020-01-03', '2020-01-15', '2020-01-27', '2020-02-08', '2020-02-20', '2020-03-03', '2020-03-15', '2020-03-27',
      '2020-04-08', '2020-04-20', '2020-05-02', '2020-05-14', '2020-05-26', '2020-06-07', '2020-06-19', '2020-07-01',
      '2020-07-13', '2020-07-25', '2020-08-06', '2020-08-18', '2020-08-30', '2020-09-11', '2020-09-23', '2020-10-05',
      '2020-10-17', '2020-10-29', '2020-11-10', '2020-11-22', '2020-12-04', '2020-12-16', '2020-12-28']

d2 = ['2020-01-04', '2020-01-16', '2020-01-28', '2020-02-09', '2020-02-21', '2020-03-04', '2020-03-16', '2020-03-28',
      '2020-04-09', '2020-04-21', '2020-05-03', '2020-05-15', '2020-05-27', '2020-06-08', '2020-06-20', '2020-07-02',
      '2020-07-14', '2020-07-26', '2020-08-07', '2020-08-19', '2020-08-31', '2020-09-12', '2020-09-24', '2020-10-06',
      '2020-10-18', '2020-10-30', '2020-11-11', '2020-11-23', '2020-12-05', '2020-12-17', '2020-12-29']


task_id = []
n = len(d1)

for idx in range(n):
    
    date1 = d1[idx]
    date2 = d2[idx]
    
    
    dataset = (ee.ImageCollection('ECMWF/ERA5/DAILY')
               .filter(ee.Filter.date(date1, date2))
               .filterBounds(small_boundary.geometry()))
    u_wind = dataset.select('u_component_of_wind_10m').first().clip(small_boundary).rename('u_wind_10m')
    v_wind = dataset.select('v_component_of_wind_10m').first().clip(small_boundary).rename('v_wind_10m')
    img = u_wind.addBands(ee.Image([v_wind]))


    # export era5 wind data
    txt = 'era5_winds_nanda_bet_' + date1
    task = ee.batch.Export.image.toDrive(image=img,
                                         description=txt,
                                         scale=27830,
                                         fileNamePrefix=txt,
                                         crs='EPSG:4326',
                                         fileFormat='GeoTIFF',
                                         region=small_boundary.geometry())
    task.start()
    task_id.append(task.id)
    
    
    print(date1)

2020-01-27
2020-02-08
2020-02-20
2020-03-03
2020-03-15
2020-03-27
2020-04-08
2020-04-20
2020-05-02
2020-05-14
2020-05-26
2020-06-07
2020-06-19
2020-07-01
2020-07-13
2020-07-25
2020-08-06
2020-08-18
2020-08-30
2020-09-11
2020-09-23
2020-10-05
2020-10-17
2020-10-29
2020-11-10
2020-11-22
2020-12-04
2020-12-16
2020-12-28


In [5]:
print(task_id)

['WBA3HZEIQNCKVN4ERFMKHXGK', 'UNF4G3YNYRC6IJA3F26EWZ3O']


In [11]:
task.list()

[<Task HQONXCFX4E5EJ4YPARR7S6DL EXPORT_IMAGE: era5_winds_nanda_bet_2020-12-28 (FAILED)>,
 <Task XZ4TVQG3A5MH4T35F2XXAUFE EXPORT_IMAGE: era5_winds_nanda_bet_2020-12-16 (FAILED)>,
 <Task SVAHQSJQUOUHKKWWGAL2N33I EXPORT_IMAGE: era5_winds_nanda_bet_2020-12-04 (FAILED)>,
 <Task AZNRELDXGZ4MSLHAG3CTHS5F EXPORT_IMAGE: era5_winds_nanda_bet_2020-11-22 (FAILED)>,
 <Task CO3B3Z3P7TZIY4SDKUJBG6P7 EXPORT_IMAGE: era5_winds_nanda_bet_2020-11-10 (FAILED)>,
 <Task J2CIJQJG6JF7M2NRIXLMRIZ5 EXPORT_IMAGE: era5_winds_nanda_bet_2020-10-29 (FAILED)>,
 <Task OZFKBD53CATTKYQZ7SWNOA7L EXPORT_IMAGE: era5_winds_nanda_bet_2020-10-17 (FAILED)>,
 <Task GRHJY5ZXSD7WAASXBIIIK4BL EXPORT_IMAGE: era5_winds_nanda_bet_2020-10-05 (FAILED)>,
 <Task ZHWRMYE24E5OUCETZUW7YME3 EXPORT_IMAGE: era5_winds_nanda_bet_2020-09-23 (FAILED)>,
 <Task 43TINPJPBEVFNCK7OPBRSESZ EXPORT_IMAGE: era5_winds_nanda_bet_2020-09-11 (FAILED)>,
 <Task 2XTUQCSLDZO25YSUA74IH6LF EXPORT_IMAGE: era5_winds_nanda_bet_2020-08-30 (FAILED)>,
 <Task KEJCILCQ5AKLIC

In [10]:
print(ee.data.getTaskStatus('HSRUNVXC6MMNBS37WXX3OIRS'))

[{'state': 'FAILED', 'description': 'era5_winds_nanda_bet_2020-07-13', 'creation_timestamp_ms': 1669761179566, 'update_timestamp_ms': 1669761279527, 'start_timestamp_ms': 1669761279274, 'task_type': 'EXPORT_IMAGE', 'attempt': 1, 'error_message': "Image.clip: Parameter 'input' is required.", 'id': 'HSRUNVXC6MMNBS37WXX3OIRS', 'name': 'projects/earthengine-legacy/operations/HSRUNVXC6MMNBS37WXX3OIRS'}]
